In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn import cluster
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from sklearn import preprocessing

# I. Import Data

In [2]:
or_dat = pd.read_csv('data/orders_dataset.csv',sep=',')
or_item = pd.read_csv('data/order_items_dataset.csv',sep=',')
cust = pd.read_csv('data/customers_dataset.csv',sep=',')
or_pay = pd.read_csv('data/order_payments_dataset.csv',sep=',')
or_review = pd.read_csv('data/order_reviews_dataset.csv',sep=',')
products = pd.read_csv('data/products_dataset.csv',sep=',')
pro_category= pd.read_csv('data/product_category_name_translation.csv',sep=',')
geo = pd.read_csv('data/geolocation_dataset.csv',sep=',')
sell =pd.read_csv('data/sellers_dataset.csv',sep=',')

In [ ]:
or_dat.head()

In [ ]:
cust.head()

In [ ]:
or_item.head()

# II. Pre-Processing
----
The first step in this processing is *Join Data*. To facilitate analysis, the data is first combined into a dataset.

## 1. Join Data

This step begins with first analyzing the variables contained in each dataset.

In [ ]:
or_dat.info()

In [ ]:
cust.info()

In the *customer* dataset there are five variables, which have the following roles:


1. customer_id: id number of each customer
2. customer_unique_id: unique id number of each customer
3. customer_zip_code_prefix: the first three digits of the customer's zip code address
4. customer_city: city of the customer's address
5. customer_state: the state of the customer's address

In [8]:
or_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In the *product* dataset there are seven variables, which have the following roles:


1. order_id: id number of each order
2. order_item_id: id number of each item in an order
3. product_id: id number of each product
4. seller_id: id number of each seller
5. shipping_limit_date: time limit until order delivery
6. price: total order price for each order
7. freight_value: the price of shipping goods for each order

Based on the variables loaded in the dataset. Therefore, to combine *order* dataset with *customer* variable *customer_id* is used.

In [ ]:
order_cust = pd.merge(or_dat,cust,on='customer_id',how='left') 
order_cust.head()

In [ ]:
df = pd.merge(order_cust,or_item, on='order_id',how="left")
df.head()

## 2. Removing Variabel

In [ ]:
df.info()

In [12]:
"Since 'customer_id', 'seller_id', 'shipping_limit_date', and 'customer_zip_code_prefix' are not usable, we will have to delete them."
df = df.drop(['customer_id', 'seller_id','shipping_limit_date','customer_zip_code_prefix'], axis = 1) 

In [ ]:
df=df[df['order_status']=='delivered']
df.head()

In [14]:
df.drop(['order_status'], axis = 1, inplace=True)

## 3. Data Transformation

In [ ]:
df.info()

In [16]:
df.iloc[:, 1:6] = df.iloc[:, 1:6].apply(pd.to_datetime, errors='coerce')

Also, even though *order_item_id* is a number, that number is just a code. Therefore, *order_item_id* is supposed to be *string* data type.

In [17]:
#column 'order_item_id' will also be changed, each will be converted to a string
df['order_item_id'] = df['order_item_id'].astype(str)

Now the data types used in this analysis are as follows.

In [ ]:
df.info()

In [ ]:
df.head()

## 4. Duplicate Removal

The next step, check whether there is data duplication in this dataset.

In [ ]:
#checking the number of rows that are the same (duplicate data)
df.duplicated().sum()

## 5. Noise Detection

Then, the characteristics of the dataset are seen and analyzed whether there is data showing values that are outside the characteristics. This can show that the data is *noise*.

In [ ]:
# Viewing descriptive statistics of the data to identify outliers (unreasonable data)
df.describe()
# The minimum value of 'freight_value' is observed to be 0, indicating a possibility of 
# free shipping. Other values seem reasonable, suggesting no apparent outliers.

Apparently, through this analysis there was no data showing characteristics that were 'strange' or different on their own.

## 6. Missing Value Detection


In [ ]:
df.isnull().sum()

It can be seen that the *order_approved_at* variable contains a missing value indicating that a purchase is not approved by Olist. Meanwhile, *missing value* in *order_delivered_carrier_date* and *order_delivered_customer_date* indicates product arrival information that was not inputted into the dataset.

In [ ]:
#percentage of missing data for each categorical and numeric variable in the data
(df.isnull().sum()/len(df)).to_frame('persentase missing')

It can be seen that the percentage of missing data in this variable is very small

In [ ]:
#A heatmap representation of missing values to visualize the distribution of missing values.
sns.heatmap(df.isnull(), cbar=False)

The distribution of *missing value* seems random, so the missing value will be dropped with the condition: if there is at least one missing value in one of the columns, then the row is deleted

In [25]:
df.dropna(axis = 0, how = 'any', inplace = True)

In [ ]:
df.isnull().sum()

In [27]:
#size (rows, columns) of the data that is ready to be used.
df.shape

(110173, 13)

Based on this processing, now obtained data that is ready to be analyzed with 13 variables and 110173 rows.

## 7. Data Visualization

In the following sections, several aspects that can be used as information to increase their income are analyzed.

### A. Number of Purchases on *E-Commerce*

#### a. Trend of Number of Purchases in 2016 - 2018

In [ ]:
# Extracting year, month, year-month, day of the week, and period from the 'order_purchase_timestamp' column
df['purchase_year'] = df['order_purchase_timestamp'].dt.strftime('%Y')  
df['purchase_month'] = df['order_purchase_timestamp'].dt.strftime('%b') 
df['purchase_yearmonth'] = df['order_purchase_timestamp'].dt.strftime('%Y%m')
df['purchase_dayofweek'] = df['order_purchase_timestamp'].dt.strftime('%a')  

# Creating a new column 'purchase_period' by dividing the hour of the purchase into 6 periods
df['purchase_period'] = (df['order_purchase_timestamp'].dt.hour % 24 + 4) // 4
df['purchase_period'].replace({
    1: 'Late Night',
    2: 'Early Morning',
    3: 'Morning',
    4: 'Noon',
    5: 'Evening',
    6: 'Night'
}, inplace=True)

# Converting month names in 'purchase_month' to corresponding month numbers
df['month_num'] = df['purchase_month'].apply(lambda x: datetime.datetime.strptime(x, "%b").month)

df.head()

In [ ]:
#plot of purchase trends from 2016 to 2018 using a line plot.
sns.set(rc={'figure.figsize':(20,5)}) 
sns.lineplot(data=df['purchase_yearmonth'].value_counts().sort_index()[3:], 
             color='darkslateblue', linewidth=2)
plt.title('Trend of Purchase Quantity from 2017 to 2018.') 

From the *line plot* above it can be seen that the sales trend
continues to increase even though there were some slight declines


In [30]:
cols = ['purchase_year', 'purchase_yearmonth'] 
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1) 

#### b. Tendency of Customers to Buy on *E-Commerce*

In [ ]:
#Bar plot of the number of purchases per day.
order=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
sns.set(rc={'figure.figsize':(20,5)})
sns.countplot(df['purchase_dayofweek'],data=df,palette='YlGnBu',order=order)
plt.title('Comparison of Purchase Quantity in a Day')
plt.xlabel('Day') 
plt.ylabel('Number of Purchases')

It can be seen that the highest number of purchases is on Monday, while the least number of purchases is on Saturday

In [32]:
#To view the total number of purchases per day.
df['purchase_dayofweek'].value_counts()

Mon    17973
Tue    17857
Wed    17217
Thu    16431
Fri    15693
Sun    13126
Sat    11876
Name: purchase_dayofweek, dtype: int64

#### c. Tendency when customers buy on *E-Commerce*

In [ ]:
#Create a bar plot of the number of purchases within 6 time periods throughout the day
order=['Early Morning', 'Morning', 'Noon', 'Evening', 'Night', 'Late Night']
sns.countplot(df['purchase_period'],data=df,palette='YlGnBu',order=order)
plt.title('Comparison of Purchase Quantity per Time Period Within a Day.') 
plt.xlabel('Time Period') 
plt.ylabel('Total Purchase') 

In [34]:
# to see the total number of purchases in 6 periodic times a day 
df['purchase_period'].value_counts()

Noon             28506
Evening          27279
Night            24432
Morning          22834
Late Night        4792
Early Morning     2330
Name: purchase_period, dtype: int64

#### d. Comparison of Number of Purchases Per Month in 2017 and 2018

In [ ]:
#A bar chart comparing the number of purchases per month in the years 2017 and 2018.
df_compare = df.query('purchase_year in (2017, 2018) &  month_num <= 8')
year_orders = df_compare['purchase_year'].value_counts()
growth = int(round(100 * (1 + year_orders[2017] / year_orders[2018]), 0))
order=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug']
sns.countplot(df_compare['purchase_month'],data=df_compare,hue='purchase_year',palette='YlGnBu',order=order)
plt.title('Perbandingan Jumlah Pembelian Per Bulan pada Tahun 2017 dan 2018')
plt.xlabel('Bulan')
plt.ylabel('Jumlah Pembelian')

From the following *bar plot*, it can be seen that the number of purchases in 2018 was greater than in 2017 every month.
Sales efforts carried out by e-commerce  were quite optimal in 2018 compared to 2017.

In [36]:
#To view the total purchase amount per month in the year 2017.
df_compare[df_compare['purchase_year']==2017]['purchase_month'].value_counts()

Aug    4797
Jul    4416
May    4003
Jun    3489
Mar    2897
Apr    2569
Feb    1845
Jan     911
Name: purchase_month, dtype: int64

In [37]:
#To see the total purchase amount per month in the year 2018.
df_compare[df_compare['purchase_year']==2018]['purchase_month'].value_counts()

Jan    8037
Mar    8017
Apr    7827
May    7810
Feb    7518
Aug    7142
Jun    7007
Jul    6960
Name: purchase_month, dtype: int64

In [38]:
#### e. Number of Purchases by City

In [39]:
df_city = df.groupby(by='customer_city', as_index=False).agg({'order_id':'count'}).sort_values(by='order_id', ascending=False).reset_index(drop=True) 
df_city_big = df_city.head() 
df_city_small = df_city.tail(20) 

In [ ]:
#Plot the number of purchases based on the city.
sns.barplot(y='customer_city', x='order_id', data=df_city_big, palette='magma')
plt.title('Total Purchases for the Top 5 Cities')
plt.xlabel('Quantity') 
plt.ylabel('City') 

From the *plot* below, it can be seen that the city of Sao Paulo has the highest number of purchases while the city of Sao Bernardo do Campo has the least number of purchases.

In [ ]:
#to see the total number of purchases for the 5 highest cities
df_city.sort_values('order_id',ascending=False)

#### f. Number of Customers Based on State

In [ ]:
#Plot the number of customers based on state
sns.set(rc={'figure.figsize':(10,10)})
sns.countplot(y=df['customer_state'], data=df, palette='viridis',order = df['customer_state'].value_counts().iloc[:5].index) 
plt.title('Number of Customers for the Top 5 States')
plt.xlabel('Number')
plt.ylabel('State')

Based on the picture above, the largest number of customers come from Sao Paulo

In [ ]:
#Plot the number of customers based on state.
sns.set(rc={'figure.figsize':(10,10)}) 
sns.countplot(y=df['customer_state'], data=df, palette='viridis',order = df['customer_state'].value_counts().iloc[-5:].index)
plt.title('Number of Customers for the 5 Lowest States')
plt.xlabel('Quantity') 
plt.ylabel('State') 

Based on the picture above, the least number of *customers* comes from Roraima(RR)

In [ ]:
#to see the total number of customers for the 5 lowest states
df['customer_state'].value_counts().iloc[-5:]

In [ ]:
df['customer_state'].value_counts().iloc[:10]

### B.Economy *E-Commerce*

#### a. Trend of Total Income

Revenue is the total cost of ordering along with shipping costs.

In [ ]:
df_month = df.groupby(by=['purchase_year', 'purchase_yearmonth'], as_index=False).agg({
    'order_id': 'count',
    'price': 'sum',
    'freight_value': 'sum'}) 
df_month['price_per_order'] = df_month['price'] / df_month['order_id'] 
df_month['freight_per_order'] = df_month['freight_value'] / df_month['order_id'] 
df_month['purchase_yearmonth']=df_month['purchase_yearmonth'].astype('str') 
df_month.head()

In [ ]:
# Large-scale purchase plot for each month from 2017 to 2018
sns.set(rc={'figure.figsize':(15,5)}) # Set plot size
plt.xticks(rotation=90) # Rotate each label on the x-axis by 90 degrees left
sns.lineplot(x='purchase_yearmonth', y='price', data=df_month[3:], linewidth=1, 
             color='darkslateblue', marker='o', label='Purchase Amount')
plt.title('Monthly Purchase Trends from 2017 to 2018') # Set the plot title
plt.xlabel('Year-Month of Purchase') # Label for the x-axis
plt.ylabel('Purchase Price') # Label for the y-axis

Overall, the trend of income has been increasing for each month although there was a significant decrease in December 2017 and income started to increase slightly in April 2018

#### b. Purchase Amount

In [ ]:
# Bar plot to visualize the number of purchases each month from 2017 to 2018
plt.xticks(rotation=90)  # To rotate each label on the x-axis by 90 degrees to the left
sns.countplot(df[df['purchase_year'] != 2016]['purchase_yearmonth'], data=df, palette='YlGnBu_r')  # To rotate x-axis labels by 90 degrees
plt.title('Number of Purchases Each Month from 2017 to 2018')  # To provide a title for the plot
plt.xlabel('Year-month of purchase')  # To label the x-axis
plt.ylabel('Number of Purchases')  # To label the y-axis


From the bar plot above, it can be seen that the amount of income has increased until November 2017 even though it decreased in December 2017. Then the amount of income was evenly distributed after December 2017

#### c. Total Purchase Price Based on State

In [ ]:
# Bar chart to visualize the total purchase price based on state
sns.set(rc={'figure.figsize':(10,10)}) # plot size
df.groupby('customer_state').agg({'price':'sum'}).sort_values('price').plot(kind='barh') # calculate the total purchase price for each state
plt.title('Total Purchase Price for Each State') # set plot title
plt.xlabel('Total Purchase Price') # set x-axis label
plt.ylabel('State') # set y-axis label


It can be seen that the highest total purchase price comes from *state* Sao Paulo(SP) while the lowest total purchase price comes from *state* Roraima(RR)

#### d. Average Price by State

In [ ]:
# Bar chart to visualize the average product prices based on state
sns.set(rc={'figure.figsize':(10,10)}) # Plot size
df.groupby('customer_state').agg({'price':'mean'}).sort_values('price').plot(kind='barh') # Calculate the average purchase price for each state
plt.title('Average Purchase Price for Each State') # Title for the plot
plt.xlabel('Average Purchase Price') # X-axis label
plt.ylabel('State') # Y-axis label


It can be seen that *state* Paraiba (PB) has the highest average purchase price, while *state* Sao Paulo (SP) has the lowest average purchase price

In [ ]:
df.groupby('customer_state').agg({'price':'mean'}).sort_values('price',ascending=False)

#### e. Trend of Average Shipping Prices

In [ ]:
# Line plot to visualize the average shipping cost per purchase each month from 2017 to 2018
plt.xticks(rotation=90) # Rotate x-axis labels by 90 degrees
sns.lineplot(x='purchase_yearmonth', y='freight_per_order', data=df_month[3:], linewidth=2, color='black', marker='o')
plt.title('Trend of Average Shipping Cost per Purchase Each Month from 2017 to 2018') # Set title for the plot
plt.xlabel('Year-Month of Purchase') # Set label for x-axis
plt.ylabel('Average Shipping Cost per Purchase') # Set label for y-axis


From the chart above, the average trend of shipments per order has increased in February 2017 and evenly thereafter until February 2018. Then as a whole, after February 2018 there has been a large increase despite experiencing several small decreases and a large decrease in August 2018

#### f. Total Shipping Prices Based on State

In [ ]:
# Bar chart to visualize the total shipping cost based on state
sns.set(rc={'figure.figsize':(10,10)}) # Plot size
df.groupby('customer_state').agg({'freight_value':'sum'}).sort_values('freight_value').plot(kind='barh') # Calculate the total shipping cost for each state
plt.title('Total Shipping Cost for Each State') # Set the title for the plot
plt.xlabel('Total Shipping Cost') # Label for the x-axis
plt.ylabel('State') # Label for the y-axis

The largest total shipping price is shipping to Sao Paulo(SP), while the lowest is to Roraima(RR)

In [ ]:
df.groupby('customer_state').agg({'freight_value':'sum'}).sort_values('freight_value')

#### h.Delivery Analysis

In [55]:
purchasing = df['order_purchase_timestamp']
delivered = df['order_delivered_customer_date']
estimated = df['order_estimated_delivery_date']
df['time_to_delivery'] = delivered - purchasing  # to calculate the duration of delivery time
df['diff_estimated_delivery'] = delivered - estimated  # to calculate the difference between actual and estimated delivery time

In [56]:
df['time_to_delivery'] = df['time_to_delivery'].apply(lambda x: x/ np.timedelta64(1, 'h')) # to convert the 'time_to_delivery' format to hours
df['diff_estimated_delivery'] = df['diff_estimated_delivery'].apply(lambda x: x/ np.timedelta64(1, 'h')) # to convert the 'diff_estimated_delivery' format to hours


In [57]:
df_state = df.groupby(by='customer_state', as_index=False).mean() # to calculate the mean of each numerical variable from the data based on state
state_freight = df_state.loc[:, ['customer_state', 'freight_value']].sort_values(by='freight_value', ascending=False) # to sort states based on shipping cost
state_time_to_delivery = df_state.loc[:, ['customer_state', 'time_to_delivery']].sort_values(by='time_to_delivery', ascending=False) # to sort states based on delivery time
state_estimated_delivery = df_state.loc[:, ['customer_state', 'diff_estimated_delivery']].sort_values(by='diff_estimated_delivery') # to sort states based on the difference between actual and estimated delivery times


In [ ]:
"To see the difference in the arrival time of a shipment from the estimated time."
state_estimated_delivery

Because all *states* have different *delivery* times and estimates are negative, meaning that **delivery to all *states* arrives earlier than the estimated estimated time**

In [ ]:
# Absolute value of the difference between delivery time and estimation will be calculated, in the column 'diff_estimated_delivery'
state_estimated_delivery['diff_estimated_delivery'] = state_estimated_delivery['diff_estimated_delivery'].abs()
state_estimated_delivery # will be printed to observe the absolute results


In [ ]:
# To visualize the top 5 states with the highest shipping costs
sns.barplot(x='freight_value', y='customer_state', data=state_freight.head(), palette='viridis')
plt.title('Shipping Costs for Top 5 States')  # To provide a title for the plot
plt.xlabel('Shipping Costs')  # To label the x-axis
plt.ylabel('State')  # To label the y-axis

It can be seen that the highest shipping price is the shipping price to Paraiba(PB)

In [ ]:
state_freight[['customer_state','freight_value']].sort_values('freight_value',ascending=False)[:5]

In [ ]:
sns.barplot(x='freight_value', y='customer_state', data=state_freight.tail(), palette='viridis_r')
plt.title('Shipping Prices for the 5 Lowest States') #to provide a title for the plot
plt.xlabel('Shipping Prices') #for labeling the x-axis
plt.ylabel('State') #for labeling the y-axis


While the lowest shipping price is the shipping price to Sao Paulo(SP)

In [ ]:

state_freight[['customer_state','freight_value']].sort_values('freight_value',ascending=False).tail()

In [ ]:
# To view the 5 states with the longest delivery times
sns.barplot(x='time_to_delivery', y='customer_state', data=state_time_to_delivery.head(), palette='viridis')
plt.title('Delivery Time for 5 States with the Longest Delays')  # To give a title to the plot
plt.xlabel('Delivery Time')  # Label for the x-axis
plt.ylabel('State')  # Label for the y-axis


In [ ]:

state_time_to_delivery[['customer_state','time_to_delivery']].sort_values('time_to_delivery',ascending=False).head()

In [ ]:
# To visualize the top 5 states with the longest delivery times
sns.barplot(x='time_to_delivery', y='customer_state', data=state_time_to_delivery.tail(), palette='viridis_r')
plt.title('Delivery Time for Top 5 States with Longest Delivery')  # To give a title to the plot
plt.xlabel('Delivery Time Duration')  # Label for the x-axis
plt.ylabel('State')  # Label for the y-axis

Meanwhile *state* Sao Paulo(SP) has the fastest delivery time

In [ ]:
#to see the delivery time for the 5 fastest states (in hours)
state_time_to_delivery[['customer_state','time_to_delivery']].sort_values('time_to_delivery',ascending=False).tail()

In [ ]:
# To visualize the 5 states with the largest difference in arrival time and estimated time
sns.barplot(x='diff_estimated_delivery', y='customer_state', data=state_estimated_delivery.head(), palette='viridis')
plt.title('5 States with the Largest Difference in Arrival Time and Estimated Time') # To provide a title for the plot
plt.xlabel('Time Difference between Delivery and Estimated Time') # Label for the x-axis
plt.ylabel('State') # Label for the y-axis

In [ ]:
state_estimated_delivery[['customer_state','diff_estimated_delivery']].sort_values('diff_estimated_delivery',ascending=False).head()

In [ ]:
# To visualize the 5 states with the smallest differences between arrival time and estimated time
sns.barplot(x='diff_estimated_delivery', y='customer_state', data=state_estimated_delivery.tail(), palette='viridis_r')
plt.title('5 States with the Smallest Differences Between Arrival Time and Estimated Time')  # To provide a title for the plot
plt.xlabel('Duration of Receiving Goods and Estimated Time')  # Label for the x-axis
plt.ylabel('State')  # Label for the y-axis

In *state* Alagoas(AL) there is a late delivery time that is faster than the estimated time

In [ ]:
#To view the time difference between arrival time and estimated time for the 5 smallest states.
state_estimated_delivery[['customer_state','diff_estimated_delivery']].sort_values('diff_estimated_delivery',ascending=False).tail()

# III. Methodology (Machine Learning):
In this section, we will create a model that can perform *customer segmentation*. *Customer segmentation* itself is an *unsupervised* problem so that our data does not have a label, but later we ourselves will provide a label. Algorithm that can be used:
- RFM Sementation
- K-Means
- Agglomerative Clustering

## Data (Machine Learning):
- Unit Analysis:
  - Customer behavior: Customer Unique ID

- Features:
  - Recency: Measures the last time a customer made a transaction.

  - Frequency: Measures how often a customer makes a transaction.

  - Monetary: Measures the total transactions made by a customer.

## RFM Segmentation 
RFM stands for *Recency, Frequency* and *Monetary*. Each factor adjusts to the characteristics of *customer*. This RFM metric is an important indicator of *customer* behavior segmentation because *frequency* and *monetary* affect *customer lifetime value*, and *recency* affects *engagement rate.*

The *RFM Analysis* application on this data aims to segment *customers*, so that e-commerce can apply sales/marketing strategies that suit the characteristics of these *customers*.

## 1. Persiapan Dataset

Because what is analyzed is *customer* characteristics, the previous data is grouped based on *customer*. 


In [118]:
import pandas as pd
from datetime import datetime, timedelta

# Modeling
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# Evaluation
from sklearn.metrics import silhouette_score

#Scaling
from sklearn.preprocessing import StandardScaler

# Regular Expression
import re

# Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df1 = pd.merge(df,or_pay,on='order_id',how='left') 
df1.head()

In [74]:
df1 = df1.drop(['payment_installments', 'payment_sequential'], axis = 1)

In [75]:
# Recency
now = datetime.now()
df1['order_purchase_timestamp'] = pd.to_datetime(df1['order_purchase_timestamp'])
df1['recency'] = (now - df1['order_purchase_timestamp']).dt.days

# Frequency
data_frequency = df1.groupby('order_item_id').order_purchase_timestamp.count()
data_frequency = data_frequency.reset_index()
data_frequency.columns = ['order_item_id','frequency']
df1 = df1.merge(data_frequency, on='order_item_id')

# Monetary
data_monetary = df1.groupby('order_item_id').payment_value.sum()
data_monetary = data_monetary.reset_index()
data_monetary.columns = ['order_item_id','monetary']
df1 = df1.merge(data_monetary, on='order_item_id')



In [ ]:
df1.columns

In [77]:
# Group data by customer_unique_id
rfm = df1.groupby(['customer_unique_id']).agg({
    'order_purchase_timestamp': lambda x: (df1['order_purchase_timestamp'].max() - x.max()).days,
    'order_id': 'count',
    'price': 'sum'
})

In [78]:
# Rename columns
rfm.rename(columns={'order_purchase_timestamp': 'recency',
                   'order_id': 'frequency',
                   'price': 'monetary'}, inplace=True)


In [79]:
# Assign score
rfm['recency_score'] = pd.qcut(rfm['recency'], q=4, labels=list(range(4, 0, -1)))
rfm['frequency_score'] = pd.qcut(rfm['frequency'], q=4, labels=list(range(4, 5)), duplicates='drop')
rfm['monetary_score'] = pd.qcut(rfm['monetary'], q=4, labels=list(range(1, 5)), duplicates='drop')

# Create RFM_Score
rfm['RFM_Score'] = rfm[['recency_score', 'frequency_score', 'monetary_score']].sum(axis=1)


In [80]:
# For most use cases, people usually look at one year data
rfm = rfm[rfm['recency'] <= 365]

In [ ]:
rfm.info()

## We'll do 5x5x5 RFM analysis

In [ ]:
# Get the sense of the distribution
rfm.quantile([.2, .4, .6, .8]).to_dict()

#### For the convenience of marketers, we can choose the grouping method as follows:

- Recency:

|  Days  | R-score |
|:------:|:-------:|
| 0-14   |    5    |
| 15-30  |    4    |
| 31-60  |    3    |
| 60-180 |    2    |
| > 180  |    1    |

- Frequency

|Frequency| F-score |
|:-------:|:-------:|
|  > 6    |    5    |
|  5 - 6  |    4    |
|  3 - 4  |    3    |
|    2    |    2    |
|    1    |    1    |


- Monetary

| Monetary   | M-score |
|:----------:|:-------:|
|   >2000    |    5    |
| (1000-2000]|    4    |
| (500-1000] |    3    |
| (250-500]  |    2    |
| (0, 250]   |    1    |


In [83]:
def r_score(x):
    if x <= 14:
        return 5
    elif x <= 30:
        return 4
    elif x <= 60:
        return 3
    elif x <= 180:
        return 2
    else:
        return 1

def f_score(x):
    if x <= 1:
        return 1
    elif x <= 2:
        return 2
    elif x <= 4:
        return 3
    elif x <= 6:
        return 4
    else:
        return 5

def m_score(x):
    if x <= 250:
        return 1
    elif x <= 500:
        return 2
    elif x <= 1000:
        return 3
    elif x <= 2000:
        return 4
    else:
        return 5   

In [84]:
rfm['R_score'] = rfm['recency'].apply(r_score)
rfm['F_score'] = rfm['frequency'].apply(f_score)
rfm['M_score'] = rfm['monetary'].apply(m_score)
rfm['RFM_score'] = rfm['R_score'] * 100 + rfm['F_score'] * 10 + rfm['M_score']

In [ ]:
# Take a peak of our RFM table by CustomerID
rfm.head()

In [86]:
# Convert the RFM_score into string
rfm['RFM_score'] = rfm['RFM_score'].astype('str')

In [ ]:
# F-R plot: Count the number of users in each F-R pair.
rfm_count = pd.pivot_table(rfm, values='RFM_score', index= ['F_score'],
                    columns=['R_score'], aggfunc='size', fill_value=0)
rfm_count

In [ ]:
# F-R plot: Find the monetary median of users in each F-R pair.
rfm_median = pd.pivot_table(rfm, values='monetary', index= ['F_score'],
                    columns=['R_score'], aggfunc='median', fill_value=0).round(2)
rfm_median

In [ ]:
# F-R plot: Find the monetary mean of users in each F-R pair.
rfm_mean = pd.pivot_table(rfm, values='monetary', index= ['F_score'],
                    columns=['R_score'], aggfunc='mean', fill_value=0).round(2)
rfm_mean

In [ ]:
# F-R plot: Find the monetary sum of users in each F-R pair.
rfm_sum = pd.pivot_table(rfm, values='monetary', index= ['F_score'],
                    columns=['R_score'], aggfunc='sum', fill_value=0).round(0).astype(int)
rfm_sum

In [ ]:
# See the RFM scores, and monetary metrics from all 5x5x5 RFM groups
rfm_agg = rfm.groupby('RFM_score').agg({'RFM_score':'size', 'monetary': ['sum', 'mean','median']}).round(2).astype(int)
rfm_agg

In [92]:
def heatmap(data, ax = None):
    
    # A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  
    # If not provided, use current axes or create a new one.
    if not ax:
        ax = plt.gca()
        
    im = ax.imshow(data, cmap="YlGn")
    
    # Show all ticks
    ax.set_xticks(np.arange(5))
    ax.set_yticks(np.arange(5))
    # label all ticks with the respective list entries
    ax.set_xticklabels(data.columns)
    ax.set_yticklabels(data.index)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Normalize the threshold to the images color range.
    threshold = im.norm(data.max().max())/2.
    textcolors=("black", "white")

    # Loop over data dimensions and create text annotations.
    for i in range(5):
        for j in range(5):
            text = ax.text(j, i, data.iloc[i, j],
                           ha="center", va="center", color=textcolors[int(im.norm(data.iloc[i, j]) > threshold)])

    # ax.set_title("RFM Customer Group Counts")
    plt.xlabel("R Score")
    plt.ylabel("F Score")
    
    return im

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=120)
heatmap(rfm_count, ax = ax)
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=120)
heatmap(rfm_median, ax = ax)
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=120)
heatmap(rfm_mean, ax = ax)
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=120)
heatmap(rfm_sum, ax = ax)
fig.tight_layout()
plt.show()

In [97]:
segmentation = {
    r'[2-5][4-5]': 'VIPs',
    r'[2-5]3': 'Potential loyalists',
    r'[1-5]2': 'Need to focus',
    r'1[3-5]': 'Good old frends',
    r'[4-5]1': 'New customers',
    r'[1-3]1': 'Hibernating',
}

In [98]:
rfm['Segment'] = rfm['R_score'] * 10 + rfm['F_score']
rfm['Segment'] = rfm['Segment'].astype(str)

In [ ]:
rfm['Segment'] = rfm['Segment'].replace(segmentation, regex=True)
rfm.head()

In [ ]:
rfm_segment = rfm.groupby('Segment').agg({'recency': ['mean', 'median'], 
                                          'frequency': ['mean', 'median'], 
                                          'monetary': ['mean', 'median', 'sum', 'count']}).round(0).astype(int)
rfm_segment

In [ ]:
segment_counts = rfm_segment['monetary']['count'].sort_values(ascending = True)
segment_counts

In [ ]:
segment_monetary_sum = rfm_segment['monetary']['sum'].sort_values(ascending = True)
segment_monetary_sum

In [103]:
def hbar(data):
    
    num_of_segment = len(data)
    
    ax.set_frame_on(False)
    ax.set_yticks(range(num_of_segment))
    ax.set_yticklabels(data.index)

    bars = ax.barh(range(num_of_segment), data, color='silver')

    for i, bar in enumerate(bars):
            value = bar.get_width()
            if data.index[i] in ['VIPs']:
                bar.set_color('firebrick')
            ax.text(value,
                    bar.get_y() + bar.get_height()/2,
                    '{:,} ({:}%)'.format(int(value),int(value*100/data.sum())),
                    va='center',
                    ha='left')

In [ ]:
fig, ax = plt.subplots()
hbar(segment_counts)
plt.show()

In [ ]:
fig, ax = plt.subplots()
hbar(segment_monetary_sum)
plt.show()

#### Calucalate Silhoutte Score

In [111]:
# Define x
x = rfm[['recency', 'frequency', 'monetary']]
# Scale
scaler = StandardScaler()
# Data Scale
x_scaled = scaler.fit_transform(x)
# Calculate Silhoutte Score
labels = rfm['RFM_Score']
silhoutte_value = silhouette_score(x_scaled, labels)
print('Silhoutte Score for RFM:', silhoutte_value)

Silhoutte Score for RFM: -0.11443209777837651


In [116]:
# Scale
scaler = StandardScaler()

# Data Scale
x_scaled = scaler.fit_transform(x)

In [119]:
score = []
number_of_clusters = range(2,9)

for i in number_of_clusters:
  kmeans = KMeans(i)
  kmeans.fit(x_scaled)

  label = kmeans.labels_
  score.append(silhouette_score(x_scaled, label, metric = 'euclidean'))

In [ ]:
for i in range(len(score)):
  print(f'Silhouette Score for {i+2} Clusters:', round(score[i], 3))

In [ ]:
plt.figure(figsize = (12, 7))

sns.lineplot(number_of_clusters, score)
sns.scatterplot(number_of_clusters, score)

plt.xlabel("Number Of Cluster")
plt.ylabel('Silhoutte Score')

In [122]:
kmeans = KMeans(n_clusters=5, random_state=10)
kmeans.fit(x_scaled)

rfm['label k-means'] = kmeans.labels_

In [ ]:
label = rfm['label k-means']
score = silhouette_score(x_scaled, label, metric = 'euclidean')
score

In [124]:
def label(x):
  if x == 0:
    return 'New Customer'
  elif x == 1:
    return 'Hibernating'
  elif x == 2:
    return 'Need to focus'
  elif x == 3:
    return 'Potential loyalists'
  else:
    return 'VIP'

rfm['label k-means'] = rfm['label k-means'].apply(label)


In [ ]:
#Number of customers in each segment
rfm['label k-means'].value_counts()

In [ ]:
## Visualisation
plt.figure(figsize = (6,4))
sns.countplot(x='label k-means', data = rfm)
plt.show()

In [ ]:
plt.figure(figsize =(18,5))

# Recency vs Frequency
plt.subplot(131)
sns.scatterplot(x = 'recency', y = 'frequency', hue = 'label k-means', data = rfm)
plt.title('Recency Vs Frequency')

# Recency Vs Monetary
plt.subplot(132)
sns.scatterplot(x = 'recency', y = 'monetary', hue = 'label k-means', data = rfm)
plt.title('Recency Vs Monetary')

# Frequency Vs Monetary
plt.subplot(133)
sns.scatterplot(x = 'frequency', y = 'monetary', hue = 'label k-means', data = rfm)
plt.title('Frequency Vs Monetary')

plt.suptitle('Model 2D Plot')
plt.show()

## Gaussian:


In [128]:
rfm = rfm[['recency', 'frequency', 'monetary']]

In [129]:
score2 = []
number_of_clusters = range(2,9)

for i in number_of_clusters:
  gsm = GaussianMixture(n_components=i)
  gsm.fit(x)

  label = gsm.predict(x)
  score2.append(silhouette_score(x_scaled, label, metric = 'euclidean'))

In [ ]:
for i in range(len(score2)):
  print(f'Silhouette Score for {i+2} Clusters', round(score2[i], 3))

In [ ]:
plt.figure(figsize = (12, 7))

sns.lineplot(number_of_clusters, score2)
sns.scatterplot(number_of_clusters, score2)

plt.xlabel("Number Of Cluster")
plt.ylabel('Silhoutte Score')

In [132]:
gsm = GaussianMixture(n_components=3)
gsm.fit(x)

rfm['label gaussian'] = gsm.predict(x)

In [ ]:
rfm['label gaussian'].value_counts()

In [ ]:
## Visualisation
plt.figure(figsize = (6,4))
sns.countplot(x='label gaussian', data = rfm)
plt.show()

In [ ]:
plt.figure(figsize =(18,5))

# Recency vs Frequency
plt.subplot(131)
sns.scatterplot(x = 'recency', y = 'frequency', hue = 'label gaussian', data = rfm)
plt.title('Recency Vs Frequency')

# Recency Vs Monetary
plt.subplot(132)
sns.scatterplot(x = 'recency', y = 'monetary', hue = 'label gaussian', data = rfm)
plt.title('Recency Vs Monetary')

# Frequency Vs Monetary
plt.subplot(133)
sns.scatterplot(x = 'frequency', y = 'monetary', hue = 'label gaussian', data = rfm)
plt.title('Frequency Vs Monetary')

plt.suptitle('Model 2D Plot')
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
eps = 0.5
min_samples = 3

# Scaling
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# DBSCAN
db = DBSCAN(eps=eps, min_samples=min_samples)
db.fit(x_scaled)

# Label
rfm['label DBSCAN'] = db.labels_

# Check Silhouette Score
score = silhouette_score(x_scaled, rfm['label DBSCAN'], metric = 'euclidean')
print('Silhouette Score for DBSCAN:', score)

# Number of Clusters
print('Number of Clusters:', len(np.unique(rfm['label DBSCAN'])))

# Visualize
plt.figure(figsize=(12, 7))

sns.scatterplot(x=rfm['recency'], y=rfm['frequency'], hue=rfm['label DBSCAN'])
plt.xlabel("Recency")
plt.ylabel('Frequency')
plt.title('DBSCAN Model 2D Plot')
plt.show()


In [ ]:
# Group by label
rfm_db = rfm.groupby('label DBSCAN')

# Calculate mean and std for each group
rfm_db_mean = rfm_db.mean()
rfm_db_std = rfm_db.std()

# Observation
# - Group 0: These customers are new to the business and have only made one purchase. They are likely to be in the early stages of the customer lifecycle and are worth further nurturing.
# - Group 1: These customers are regular customers who make multiple purchases over time. They are likely to be loyal customers and are worth retaining.
# - Group 2: These customers are infrequent customers who have made a few purchases over time. They are likely to be less engaged with the business and are worth targeting with promotional activities.
# - Group 3: These customers are inactive customers who have not made a purchase in a long time. They are likely to be lost customers and are worth re-engaging with.
# We can use this information to develop targeted marketing campaigns for each group of customers.

In [ ]:
# Import the necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_scaled, rfm['label k-means'], test_size=0.25, random_state=10)

# Create the KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


In [ ]:
from sklearn import svm
# Create a support vector machine classifier
svm_classifier = svm.LinearSVC()

# Train the classifier on the training data
svm_classifier.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = svm_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


In [ ]:
df1.info()

In [ ]:
df1.head(5)

In [ ]:
X = df1[['monetary']]
y = df1['frequency']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import sklearn.metrics

mse = sklearn.metrics.mean_squared_error(y_test, y_pred)
r2 = sklearn.metrics.r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
plt.scatter(X_test, y_test, color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)
plt.xlabel('Independent Variable')
plt.ylabel('Dependent Variable')
plt.title('Linear Regression Model')
plt.show()